In [1]:
import skimage.io as io
from skimage.color import rgb2gray
import os
import pandas as pd
from pandas.tools.plotting import table
import numpy as np
from imp import reload
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier as DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.grid_search import GridSearchCV
import sys
sys.path.append('../')
import helpers.image_processing as im
import helpers.histogram_classifier as hc

In [2]:
from IPython.core.display import HTML
css = open('styles/table-style.css').read() #+ open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
classes = pd.read_csv('../data/food-101/meta/top_classes.csv', index_col=0)
class_list = list(classes['class'].unique())

image_names = {}
for c in class_list:
    image_names[c] = np.array(classes[classes['class'] == c]['name'])

In [4]:
def get_features(class_list, image_names):
    labels = []
    data = []
    for c in class_list:
        for i in image_names[c]:
            img = io.imread(os.path.join('../data/food-101/top_classes/', c, '', i))
            hist = hc.color_histogram(img)
            labels.append(c)
            data.append(hist)
    return np.array(labels), np.array(data)

In [5]:
train, test = hc.split_data(image_names, 0.75)

In [6]:
train_labels, train_features = get_features(class_list, train)

In [7]:
test_labels, test_features = get_features(class_list, test)

***1. K Nearest Neighbors, K=10, Uniform Weighting***

In [8]:
%%time
model = KNeighborsClassifier(n_neighbors=10)
model.fit(train_features, train_labels)

CPU times: user 423 ms, sys: 21.4 ms, total: 444 ms
Wall time: 443 ms


In [9]:
%%time
predicted = model.predict(test_features)

CPU times: user 31.9 s, sys: 259 ms, total: 32.1 s
Wall time: 32.8 s


In [10]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [11]:
overall

,Accuracy,F1,Precision,Recall
Results,0.196,0.192,0.202,0.196


In [12]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.16,0.11,0.13,250.0
lasagna,0.19,0.11,0.14,250.0
french_toast,0.14,0.23,0.18,250.0
guacamole,0.29,0.29,0.29,250.0
apple_pie,0.13,0.14,0.13,250.0
cheesecake,0.26,0.14,0.18,250.0
hamburger,0.15,0.22,0.18,250.0
fried_rice,0.23,0.37,0.28,250.0
carrot_cake,0.13,0.11,0.12,250.0
chocolate_cake,0.32,0.20,0.25,250.0


In [13]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,27,13,49,10,18,9,42,9,20,11,25,17
lasagna,11,28,39,9,19,5,24,52,18,5,12,28
french_toast,16,13,58,13,22,4,34,25,17,5,6,37
guacamole,6,9,17,73,21,3,38,36,10,9,14,14
apple_pie,15,14,40,12,34,9,25,44,11,11,13,22
cheesecake,10,15,32,16,30,35,19,16,29,15,18,15
hamburger,9,11,37,26,17,6,55,34,10,6,11,28
fried_rice,4,11,25,17,20,6,40,92,5,4,7,19
carrot_cake,22,12,42,21,22,9,21,27,28,11,21,14
chocolate_cake,15,8,14,14,19,32,15,14,28,50,25,16


***2. Linear SVM***

In [143]:
%%time
model = LinearSVC()
model.fit(train_features, train_labels)

CPU times: user 3min 39s, sys: 1.4 s, total: 3min 40s
Wall time: 3min 42s


In [144]:
%%time
predicted = model.predict(test_features)

CPU times: user 13.4 ms, sys: 5.26 ms, total: 18.7 ms
Wall time: 12.7 ms


In [145]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [146]:
overall

,Accuracy,F1,Precision,Recall
Results,0.135,0.1,0.133,0.135


In [147]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.13,0.05,0.07,250.0
lasagna,0.10,0.01,0.02,250.0
french_toast,0.18,0.14,0.15,250.0
guacamole,0.18,0.14,0.16,250.0
apple_pie,0.03,0.01,0.01,250.0
cheesecake,0.19,0.08,0.11,250.0
hamburger,0.04,0.02,0.02,250.0
fried_rice,0.12,0.78,0.20,250.0
carrot_cake,0.10,0.05,0.07,250.0
chocolate_cake,0.21,0.24,0.23,250.0


In [148]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,12,5,14,13,5,8,13,118,15,33,0,14
lasagna,12,3,18,7,7,6,7,145,12,6,2,25
french_toast,4,3,34,7,1,9,7,153,9,11,1,11
guacamole,3,0,3,35,4,5,2,165,3,24,1,5
apple_pie,6,3,16,14,2,4,6,163,13,11,1,11
cheesecake,6,4,21,18,8,20,12,96,15,40,3,7
hamburger,4,2,13,22,7,2,4,161,6,17,0,12
fried_rice,6,2,4,14,2,6,3,194,4,9,0,6
carrot_cake,8,2,25,11,6,17,11,122,13,25,0,10
chocolate_cake,6,1,22,23,11,10,14,68,24,61,2,8


***3. Support Vector Machine Polynomial Kernel***

In [149]:
%%time
model = SVC(kernel="poly")
model.fit(train_features, train_labels)

CPU times: user 2min, sys: 982 ms, total: 2min 1s
Wall time: 2min 4s


In [150]:
%%time
predicted = model.predict(test_features)

CPU times: user 20.8 s, sys: 55.4 ms, total: 20.9 s
Wall time: 20.9 s


In [151]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [152]:
overall

,Accuracy,F1,Precision,Recall
Results,0.222,0.222,0.225,0.222


In [153]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.14,0.11,0.12,250.0
lasagna,0.21,0.18,0.19,250.0
french_toast,0.21,0.22,0.22,250.0
guacamole,0.29,0.31,0.30,250.0
apple_pie,0.14,0.20,0.17,250.0
cheesecake,0.21,0.24,0.22,250.0
hamburger,0.22,0.19,0.20,250.0
fried_rice,0.29,0.31,0.30,250.0
carrot_cake,0.15,0.18,0.17,250.0
chocolate_cake,0.34,0.32,0.33,250.0


In [154]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,27,18,18,16,20,21,22,12,41,17,31,7
lasagna,17,44,23,10,38,13,12,26,25,4,15,23
french_toast,16,21,55,11,22,18,23,8,30,9,15,22
guacamole,9,11,5,77,31,15,23,26,14,16,15,8
apple_pie,7,21,26,25,50,19,13,29,22,7,13,18
cheesecake,19,9,19,16,34,60,10,13,34,22,12,2
hamburger,18,20,24,27,29,10,48,21,15,6,9,23
fried_rice,8,13,18,27,29,16,16,77,15,10,8,13
carrot_cake,15,15,22,11,24,31,13,21,46,20,20,12
chocolate_cake,11,7,14,11,19,47,3,6,25,80,23,4


***4. Decision Tree***

In [155]:
%%time
model = DecisionTreeClassifier(max_depth=5)
model.fit(train_features, train_labels)

CPU times: user 3.21 s, sys: 16.6 ms, total: 3.23 s
Wall time: 3.23 s


In [156]:
%%time
predicted = model.predict(test_features)

CPU times: user 6.22 ms, sys: 2.91 ms, total: 9.13 ms
Wall time: 7.9 ms


In [157]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [158]:
overall

,Accuracy,F1,Precision,Recall
Results,0.185,0.166,0.189,0.185


In [159]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.13,0.07,0.09,250.0
lasagna,0.24,0.12,0.16,250.0
french_toast,0.17,0.09,0.12,250.0
guacamole,0.17,0.18,0.17,250.0
apple_pie,0.09,0.06,0.07,250.0
cheesecake,0.27,0.30,0.29,250.0
hamburger,0.13,0.48,0.21,250.0
fried_rice,0.28,0.37,0.32,250.0
carrot_cake,0.03,0.00,0.01,250.0
chocolate_cake,0.27,0.26,0.26,250.0


In [160]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,17,9,6,21,6,16,86,8,3,24,54,0
lasagna,9,30,22,13,12,14,80,33,1,9,17,10
french_toast,11,13,23,15,13,17,95,22,2,11,19,9
guacamole,5,6,7,45,13,17,75,37,5,15,24,1
apple_pie,10,14,14,15,14,19,76,52,2,14,15,5
cheesecake,8,5,9,15,27,75,37,14,1,36,21,2
hamburger,6,4,13,25,13,12,120,19,0,6,28,4
fried_rice,6,1,11,19,14,9,71,92,0,6,18,3
carrot_cake,16,11,13,20,20,27,55,21,1,20,46,0
chocolate_cake,11,2,5,24,5,40,36,3,11,65,48,0


***5. Random Forest***

In [47]:
%%time
model = RandomForestClassifier(max_depth=6, n_estimators=14, max_features="sqrt")
model.fit(train_features, train_labels)

CPU times: user 1.35 s, sys: 18.3 ms, total: 1.37 s
Wall time: 1.37 s


In [48]:
%%time
predicted = model.predict(test_features)

CPU times: user 17.6 ms, sys: 7.43 ms, total: 25.1 ms
Wall time: 23.7 ms


In [49]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [50]:
overall

,Accuracy,F1,Precision,Recall
Results,0.23,0.218,0.23,0.23


In [51]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.18,0.12,0.15,250.0
lasagna,0.23,0.19,0.21,250.0
french_toast,0.22,0.08,0.12,250.0
guacamole,0.24,0.32,0.27,250.0
apple_pie,0.16,0.09,0.11,250.0
cheesecake,0.32,0.29,0.30,250.0
hamburger,0.19,0.34,0.25,250.0
fried_rice,0.26,0.32,0.29,250.0
carrot_cake,0.26,0.09,0.14,250.0
chocolate_cake,0.29,0.33,0.31,250.0


In [52]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,31,8,10,31,4,10,38,12,7,23,58,18
lasagna,17,47,11,14,6,14,31,30,2,14,21,43
french_toast,10,35,21,20,15,9,40,25,8,10,29,28
guacamole,6,5,6,81,10,7,35,30,2,19,29,20
apple_pie,13,25,7,20,22,18,29,47,11,9,20,29
cheesecake,12,14,4,19,16,72,18,19,9,32,26,9
hamburger,10,22,6,27,12,6,84,19,4,9,22,29
fried_rice,5,10,6,24,17,15,43,81,4,9,17,19
carrot_cake,24,11,16,18,14,28,24,13,23,32,34,13
chocolate_cake,13,3,3,24,5,34,14,5,8,83,54,4


***6. Ada Boost Classifier***

In [30]:
%%time
model = AdaBoostClassifier(n_estimators=60, learning_rate=0.8)
model.fit(train_features, train_labels)

CPU times: user 39.3 s, sys: 271 ms, total: 39.6 s
Wall time: 39.8 s


In [31]:
%%time
predicted = model.predict(test_features)

CPU times: user 135 ms, sys: 5.64 ms, total: 140 ms
Wall time: 139 ms


In [32]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [33]:
overall

,Accuracy,F1,Precision,Recall
Results,0.223,0.217,0.22,0.223


In [34]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.19,0.21,0.20,250.0
lasagna,0.25,0.24,0.25,250.0
french_toast,0.17,0.21,0.19,250.0
guacamole,0.24,0.28,0.26,250.0
apple_pie,0.22,0.14,0.17,250.0
cheesecake,0.26,0.24,0.25,250.0
hamburger,0.21,0.22,0.22,250.0
fried_rice,0.25,0.26,0.26,250.0
carrot_cake,0.19,0.08,0.11,250.0
chocolate_cake,0.28,0.44,0.34,250.0


In [35]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,53,13,23,20,9,17,21,10,7,23,42,12
lasagna,15,61,36,9,15,11,18,24,8,13,13,27
french_toast,27,27,53,13,20,11,22,22,5,17,12,21
guacamole,11,14,12,69,13,10,19,34,3,29,23,13
apple_pie,16,28,42,21,34,17,16,21,8,18,9,20
cheesecake,10,12,10,22,9,60,11,15,16,59,17,9
hamburger,21,22,32,20,8,8,56,20,4,10,20,29
fried_rice,13,15,15,33,11,21,26,66,10,11,5,24
carrot_cake,28,19,34,16,11,25,9,13,19,36,30,10
chocolate_cake,19,2,12,16,7,27,4,4,8,109,34,8


***7. Gaussian NB***

In [173]:
%%time
model = GaussianNB()
model.fit(train_features, train_labels)

CPU times: user 117 ms, sys: 44.8 ms, total: 161 ms
Wall time: 161 ms


In [174]:
%%time
predicted = model.predict(test_features)

CPU times: user 219 ms, sys: 118 ms, total: 337 ms
Wall time: 338 ms


In [175]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [176]:
overall

,Accuracy,F1,Precision,Recall
Results,0.192,0.177,0.199,0.192


In [177]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.22,0.04,0.07,250.0
lasagna,0.16,0.04,0.07,250.0
french_toast,0.17,0.27,0.21,250.0
guacamole,0.15,0.36,0.21,250.0
apple_pie,0.23,0.08,0.12,250.0
cheesecake,0.22,0.24,0.23,250.0
hamburger,0.16,0.23,0.19,250.0
fried_rice,0.25,0.28,0.26,250.0
carrot_cake,0.15,0.05,0.07,250.0
chocolate_cake,0.25,0.21,0.23,250.0


In [178]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,11,4,41,49,0,17,30,10,10,13,40,25
lasagna,8,11,34,37,8,27,35,25,2,12,11,40
french_toast,2,6,68,32,6,16,38,13,11,12,12,34
guacamole,1,6,24,90,6,4,32,31,3,11,12,30
apple_pie,4,13,32,36,20,28,26,28,6,16,9,32
cheesecake,6,6,25,38,12,59,11,25,16,31,11,10
hamburger,0,4,34,63,4,4,58,22,0,6,16,39
fried_rice,0,8,27,46,6,17,35,69,2,5,2,33
carrot_cake,3,5,45,49,9,32,17,17,12,18,30,13
chocolate_cake,3,0,19,46,8,34,13,12,9,53,39,14


***8. Multinomial Naive Bayes***

In [181]:
%%time
model = MultinomialNB()
model.fit(train_features, train_labels)

CPU times: user 55.6 ms, sys: 11.9 ms, total: 67.5 ms
Wall time: 52.5 ms


In [182]:
%%time
predicted = model.predict(test_features)

CPU times: user 14.7 ms, sys: 5.24 ms, total: 19.9 ms
Wall time: 14.6 ms


In [183]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [184]:
overall

,Accuracy,F1,Precision,Recall
Results,0.193,0.175,0.185,0.193


In [185]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.19,0.08,0.11,250.0
lasagna,0.14,0.08,0.10,250.0
french_toast,0.16,0.11,0.13,250.0
guacamole,0.16,0.32,0.21,250.0
apple_pie,0.16,0.10,0.13,250.0
cheesecake,0.22,0.29,0.25,250.0
hamburger,0.13,0.02,0.03,250.0
fried_rice,0.21,0.38,0.27,250.0
carrot_cake,0.23,0.14,0.18,250.0
chocolate_cake,0.24,0.33,0.28,250.0


In [186]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,19,15,16,45,5,22,2,17,21,40,25,23
lasagna,10,20,17,35,15,24,3,46,8,20,13,39
french_toast,9,19,27,22,26,26,6,27,19,24,16,29
guacamole,4,7,14,81,14,14,5,47,3,15,20,26
apple_pie,7,19,11,29,26,26,1,55,8,20,13,35
cheesecake,3,3,9,38,13,72,1,34,15,42,13,7
hamburger,5,13,17,52,14,14,4,49,5,20,21,36
fried_rice,9,12,11,35,17,24,1,96,4,4,7,30
carrot_cake,13,9,13,43,13,38,2,23,36,25,18,17
chocolate_cake,3,2,11,32,6,37,2,14,13,83,34,13


***9. Bernoulli Naive Bayes***

In [189]:
%%time
model = BernoulliNB()
model.fit(train_features, train_labels)

CPU times: user 102 ms, sys: 42.7 ms, total: 145 ms
Wall time: 126 ms


In [190]:
%%time
predicted = model.predict(test_features)

CPU times: user 34.4 ms, sys: 12.9 ms, total: 47.4 ms
Wall time: 37.1 ms


In [191]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [192]:
overall

,Accuracy,F1,Precision,Recall
Results,0.104,0.067,0.119,0.104


In [193]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.09,0.04,0.06,250.0
lasagna,0.13,0.04,0.06,250.0
french_toast,0.09,0.77,0.17,250.0
guacamole,0.04,0.00,0.01,250.0
apple_pie,0.12,0.08,0.09,250.0
cheesecake,0.13,0.09,0.11,250.0
hamburger,0.15,0.02,0.03,250.0
fried_rice,0.08,0.01,0.01,250.0
carrot_cake,0.20,0.02,0.04,250.0
chocolate_cake,0.13,0.01,0.02,250.0


In [194]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,10,3,183,3,12,10,2,4,1,0,9,13
lasagna,7,10,157,3,28,11,4,2,2,2,2,22
french_toast,4,12,192,2,13,8,1,2,5,2,4,5
guacamole,11,4,181,1,12,13,2,3,1,2,8,12
apple_pie,11,11,157,2,20,11,3,2,3,2,11,17
cheesecake,11,7,157,3,17,22,1,2,1,1,12,16
hamburger,8,5,173,4,12,17,4,1,1,3,7,15
fried_rice,11,2,163,1,16,15,1,2,3,1,14,21
carrot_cake,6,8,160,2,13,16,4,3,5,3,12,18
chocolate_cake,8,5,194,2,7,4,0,2,1,3,13,11


***10. Linear Discriminant Analysis***

In [195]:
%%time
model = LinearDiscriminantAnalysis()
model.fit(train_features, train_labels)

CPU times: user 2.15 s, sys: 132 ms, total: 2.29 s
Wall time: 1.19 s


/Users/simonbedford/anaconda3/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [196]:
%%time
predicted = model.predict(test_features)

CPU times: user 16.3 ms, sys: 6.16 ms, total: 22.5 ms
Wall time: 14.5 ms


In [197]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

In [198]:
overall

,Accuracy,F1,Precision,Recall
Results,0.199,0.196,0.194,0.199


In [199]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.15,0.13,0.14,250.0
lasagna,0.14,0.13,0.13,250.0
french_toast,0.15,0.18,0.17,250.0
guacamole,0.33,0.42,0.37,250.0
apple_pie,0.14,0.11,0.12,250.0
cheesecake,0.20,0.21,0.21,250.0
hamburger,0.12,0.11,0.11,250.0
fried_rice,0.26,0.24,0.25,250.0
carrot_cake,0.13,0.12,0.13,250.0
chocolate_cake,0.28,0.28,0.28,250.0


In [200]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,33,20,17,15,11,25,15,11,32,22,32,17
lasagna,23,32,23,16,20,15,23,28,27,8,8,27
french_toast,26,35,46,10,14,18,19,10,23,10,12,27
guacamole,14,12,15,105,14,12,18,16,8,11,15,10
apple_pie,10,21,35,28,28,21,20,22,13,9,10,33
cheesecake,20,19,20,17,24,53,12,7,25,29,15,9
hamburger,13,18,35,38,18,11,27,24,10,14,19,23
fried_rice,14,14,29,30,12,12,33,59,11,8,9,19
carrot_cake,14,14,28,10,22,34,16,16,31,20,29,16
chocolate_cake,11,11,14,9,14,31,10,7,24,69,40,10


***11. Quadratic Discriminant Analysis***

In [201]:
%%time
model = QuadraticDiscriminantAnalysis()
model.fit(train_features, train_labels)

/Users/simonbedford/anaconda3/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


CPU times: user 3.81 s, sys: 87.9 ms, total: 3.9 s
Wall time: 1.99 s


In [202]:
%%time
predicted = model.predict(test_features)

CPU times: user 1.53 s, sys: 123 ms, total: 1.65 s
Wall time: 841 ms


In [203]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted, class_list)

/Users/simonbedford/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/simonbedford/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/simonbedford/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [204]:
overall

,Accuracy,F1,Precision,Recall
Results,0.091,0.056,0.118,0.091


In [205]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.09,0.05,0.06,250.0
lasagna,0.25,0.00,0.01,250.0
french_toast,0.05,0.02,0.02,250.0
guacamole,0.09,0.07,0.08,250.0
apple_pie,0.00,0.00,0.00,250.0
cheesecake,0.10,0.07,0.08,250.0
hamburger,0.12,0.11,0.11,250.0
fried_rice,0.44,0.02,0.03,250.0
carrot_cake,0.12,0.06,0.08,250.0
chocolate_cake,0.00,0.00,0.00,250.0


In [206]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,12,0,6,19,0,18,18,0,9,0,162,6
lasagna,16,1,4,22,1,12,21,0,6,0,154,13
french_toast,14,0,4,10,0,13,18,1,12,0,166,12
guacamole,7,0,12,18,2,22,19,2,10,0,151,7
apple_pie,9,1,9,17,0,13,18,1,9,0,159,14
cheesecake,16,1,9,18,0,18,17,0,7,0,150,14
hamburger,12,0,2,24,0,9,27,0,9,0,161,6
fried_rice,12,0,5,17,1,20,23,4,9,0,147,12
carrot_cake,9,0,2,12,1,8,16,0,14,0,171,17
chocolate_cake,9,0,9,13,0,10,17,0,11,0,171,10


### Grid Search Optimization

***Support Vector Machine - Polynomial Kernel***

In [53]:
degree = [2, 3, 4, 5]

In [54]:
model = SVC(kernel="poly")

In [55]:
grid = GridSearchCV(estimator=model, param_grid=dict(degree=degree))

In [ ]:
grid.fit(test_features, test_labels)

***Random Forest***

In [40]:
estimators = np.array([i for i in range(1, 16)])
depth = np.array([i for i in range(1, 16)])

In [41]:
model = RandomForestClassifier()

In [42]:
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, max_depth=depth))

In [43]:
grid.fit(test_features, test_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]), 'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [44]:
print(grid.best_score_)

0.216333333333


In [45]:
print(grid.best_estimator_.n_estimators)

14


In [46]:
print(grid.best_estimator_.max_depth)

6


***AdaBoostClassifier***

In [23]:
estimators = np.array([10, 20, 30, 40, 50, 60])
rate = np.array([0.8, 0.9, 1.0, 1.1, 1.2])

In [24]:
model = AdaBoostClassifier()

In [25]:
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, learning_rate=rate))

In [26]:
grid.fit(test_features, test_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': array([ 0.8,  0.9,  1. ,  1.1,  1.2]), 'n_estimators': array([10, 20, 30, 40, 50, 60])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [27]:
print(grid.best_score_)

0.195


In [28]:
print(grid.best_estimator_.n_estimators)

60


In [29]:
print(grid.best_estimator_.learning_rate)

0.8
